# Implementieren einer lokalen Suchmaschine

## Einführung
Im 2. Assignment werden wir einen lokalen Such-Algorithmus entwickeln, der die Informationen über die Volleyballspielerinnen des österreichischen Nationalteams effizient durchsuchen kann. Dazu werden wir zunächst die Daten des Kaders, die im 1. Assignment gesammelt wurden, verwenden.

### 1. Installation und Importieren der benötigten Libraries

In [ ]:
!pip install -r requirements.txt


In [2]:
import pandas as pd
from collections import defaultdict
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
nltk.download('punkt')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/annina/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /Users/annina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 2. Importieren der bereinigten Spielerinnendaten als CSV

Die CSV-Daten sind bereits bereinigt. In 3.2 werden die Daten nochmals bereinigt, indem strings auf lowercase() gebracht und Sonderzeichen entfernt werden.

In [3]:
spielerinnen_csv = pd.read_csv('./Spielerinnendaten.csv')
spielerinnen_csv

,Dressnummer,Name,Position,Nationalitaet,Groesse
0,1,Nicole Leonie HOLZINGER,Aufspiel,AUT,175 cm
1,2,Carmen RAAB,Diagonal,AUT,179 cm
2,3,Tamina HUBER,Libero,AUT,173 cm
3,10,Julia TRUNNER,Diagonal,AUT,185 cm
4,11,Monika CHRTIANSKA,Außenangriff,AUT,183 cm
5,13,Lina HINTEREGGER,Außenangriff,AUT,180 cm
6,14,Kora Marina SCHABERL,Außenangriff,AUT,183 cm
7,15,Anna OBERHAUSER,Libero,AUT,166 cm
8,17,Dana SCHMIT,Aufspiel,AUT,175 cm
9,18,Nina NESIMOVIC,Mittelblock,AUT,188 cm


### 3. Implementierung des Inverted Index
#### 3.1 Inhalt des CSVs in Token splitten

In [4]:
# Es werden alle Zeilen und Spalten durchlaufen, um die Inhalte in Token aufzuteilen
for index, row in spielerinnen_csv.iterrows():
    for col in spielerinnen_csv.columns:
        cell_value = str(row[col])  # Konvertierung des Zelleninhalt in einen String, falls er kein Text ist
        tokens = word_tokenize(cell_value)
        print(f"Zeile {index}, Spalte {col}: {tokens}")

# Letzer Token außerhalb der Schleife wird hier zusätzlich ausgegeben
display(tokens)


Zeile 0, Spalte Dressnummer: ['1']
Zeile 0, Spalte Name: ['Nicole', 'Leonie', 'HOLZINGER']
Zeile 0, Spalte Position: ['Aufspiel']
Zeile 0, Spalte Nationalitaet: ['AUT']
Zeile 0, Spalte Groesse: ['175', 'cm']
Zeile 1, Spalte Dressnummer: ['2']
Zeile 1, Spalte Name: ['Carmen', 'RAAB']
Zeile 1, Spalte Position: ['Diagonal']
Zeile 1, Spalte Nationalitaet: ['AUT']
Zeile 1, Spalte Groesse: ['179', 'cm']
Zeile 2, Spalte Dressnummer: ['3']
Zeile 2, Spalte Name: ['Tamina', 'HUBER']
Zeile 2, Spalte Position: ['Libero']
Zeile 2, Spalte Nationalitaet: ['AUT']
Zeile 2, Spalte Groesse: ['173', 'cm']
Zeile 3, Spalte Dressnummer: ['10']
Zeile 3, Spalte Name: ['Julia', 'TRUNNER']
Zeile 3, Spalte Position: ['Diagonal']
Zeile 3, Spalte Nationalitaet: ['AUT']
Zeile 3, Spalte Groesse: ['185', 'cm']
Zeile 4, Spalte Dressnummer: ['11']
Zeile 4, Spalte Name: ['Monika', 'CHRTIANSKA']
Zeile 4, Spalte Position: ['Außenangriff']
Zeile 4, Spalte Nationalitaet: ['AUT']
Zeile 4, Spalte Groesse: ['183', 'cm']
Zeile 5

['185', 'cm']

#### 3.2 Erstellen des Inverted Index
Jeder Token wird den entsprechenden Dokumenten zugeordnet, in denen es vorkommt, um eine strukturierte Darstellung der Daten zu erstellen.
In diesem Kontext bezieht sich "Dokumente" nicht auf physische Dokumente wie Papierseiten oder Dateien im herkömmlichen Sinne, sondern auf einzelne Textabschnitte oder Textblöcke, die durchsucht und indexiert werden. Dies könnten zum Beispiel Absätze, Sätze, Artikel, Tweets oder andere Textfragmente sein.


In [5]:
def clean_text(text):
    # Entfernt Sonderzeichen und macht alles in lowercase
    text = re.sub(r'\W+', ' ', text)
    return text.lower()

# Funktionsdefinition eines Inverted Index:
# In der Funktion build_inverted_index() wird jedem Text ein doc_id zugewiesen, das die Position des Textes in der Liste darstellt.
def build_inverted_index(docs):
    inverted_index = defaultdict(list)
    for doc_id, doc_fields in enumerate(docs):
        # Kombiniere alle Felder der Zeile zu einem einzigen String
        combined_fields = " ".join([str(field) for field in doc_fields])  # Konvertiere jedes Feld in einen String
        tokens = word_tokenize(clean_text(combined_fields))
        for token in tokens:
            if doc_id not in inverted_index[token]:
                inverted_index[str(token)].append(doc_id)  # Stelle sicher, dass der Token als String gespeichert wird
    return inverted_index


documents = spielerinnen_csv[['Dressnummer', 'Name', 'Position', 'Nationalitaet', 'Groesse']].values.tolist()

# Invertierten Index für alle Felder in Spielerinnen_CSV erstellen
index = build_inverted_index(documents)
display(index)



defaultdict(list,
            {'1': [0],
             'nicole': [0],
             'leonie': [0],
             'holzinger': [0],
             'aufspiel': [0, 8],
             'aut': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
             '175': [0, 8],
             'cm': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
             '2': [1],
             'carmen': [1],
             'raab': [1],
             'diagonal': [1, 3, 14],
             '179': [1],
             '3': [2],
             'tamina': [2],
             'huber': [2],
             'libero': [2, 7, 10],
             '173': [2],
             '10': [3],
             'julia': [3],
             'trunner': [3],
             '185': [3, 12, 14],
             '11': [4],
             'monika': [4],
             'chrtianska': [4],
             'außenangriff': [4, 5, 6],
             '183': [4, 6],
             '13': [5],
             'lina': [5],
             'hinteregger': [5],
             '180': [5],
             '14':

### 4. Erstellen des TD-IDF Scorings

### 5. Implementierung der lokalen Suchmaschine

# Ressourcen & Source-Docs

## Inverted Index
1. [Implementieren eines Inverted Index](https://www.pingcap.com/article/step-by-step-guide-building-inverted-index-python/)
2. [LinkName](https://www.google.com/)
3. [LinkName](https://www.google.com/)
4. [LinkName](https://www.google.com/)
5. [LinkName](https://www.google.com/)

## Title
1. [LinkName](https://www.google.com/)
2. [LinkName](https://www.google.com/)

## Title
1. [LinkName](https://www.google.com/)
2. [LinkName](https://www.google.com/)

# Challenges bei der Implementierung


#### Cesar Laura
-
-


#### Dilly Julian
-
-

#### Ecker Annina, Implementierung von Selenium:
- 
- 